# Use `Lale` `AIF360` scorers to calculate and mitigate bias for credit risk AutoAI model

This notebook contains the steps and code to demonstrate support of AutoAI experiments in watsonx.ai service. It introduces commands for bias detecting and mitigation performed with `lale.lib.aif360` module.

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

-  Work with watsonx.ai experiments to train AutoAI models.
-  Calculate fairness metrics of trained pipelines. 
-  Refine the best model and perform mitigation to get less biased model.
-  Store trained model with custom software specification.
-  Online deployment and score the trained model.

## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Optimizer definition](#definition)
3. [Experiment Run](#run)
4. [Pipeline bias detection and mitigation](#bias)
5. [Deployment and score](#scoring)
6. [Clean up](#cleanup)
7. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

If you are not familiar with <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">watsonx.ai</a> and AutoAI experiments, please read more about it in the sample notebook: <a href="https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/experiments/autoai/Use%20AutoAI%20and%20Lale%20to%20predict%20credit%20risk.ipynb" target="_blank" rel="noopener no referrer">"Use AutoAI and Lale to predict credit risk with `ibm-watsonx-ai`"</a>

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U wget | tail -n 1
%pip install "scikit-learn==1.6.1" | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1
%pip install -U autoai-libs | tail -n 1
%pip install -U "lale[fairness]" | tail -n 1
%pip install -U aif360 | tail -n 1
%pip install -U liac-arff | tail -n 1
%pip install -U cvxpy | tail -n 1
%pip install -U fairlearn | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

Enter your watsonx.ai password and hit enter:  ········


#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below


In [5]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use the `list` method to print all existing spaces.


In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai, you need to set the **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="definition"></a>
## 2. Optimizer definition

### Training data connection

Define connection information to training data CSV file. This example uses the [German Credit Risk dataset](https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cloud/data/credit_risk/german_credit_data_biased_training.csv).

In [7]:
filename = "german_credit_data_biased_training.csv"

Download training data from git repository and split for training and test set.

In [ ]:
import os, wget
import pandas as pd
from sklearn.model_selection import train_test_split

url = "https://raw.githubusercontent.com/IBM/watsonx-ai-samples/master/cpd5.2/data/credit_risk/german_credit_data_biased_training.csv"
if not os.path.isfile(filename):
    wget.download(url)

credit_risk_df = pd.read_csv(filename)
X = credit_risk_df.drop(["Risk"], axis=1)
y = credit_risk_df["Risk"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

credit_risk_df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


Upload train dataset as Cloud Pak for Data data asset and define connection information to training data.


In [9]:
X_train.join(y_train).to_csv(filename, index=False, mode="w+")
asset_details = client.data_assets.create(filename, filename)
asset_details

Creating data asset...
SUCCESS


{'metadata': {'space_id': '9c4eb218-96e8-41e0-8cd5-f783504a11ea',
  'usage': {'last_updated_at': '2025-05-23T14:19:52Z',
   'last_updater_id': '1000331001',
   'last_update_time': 1748009992882,
   'last_accessed_at': '2025-05-23T14:19:52Z',
   'last_access_time': 1748009992882,
   'last_accessor_id': '1000331001',
   'access_count': 0},
  'rov': {'mode': 0,
   'collaborator_ids': {},
   'member_roles': {'1000331001': {'user_iam_id': '1000331001',
     'roles': ['OWNER']}}},
  'is_linked_with_sub_container': False,
  'name': 'german_credit_data_biased_training.csv',
  'description': '',
  'asset_type': 'data_asset',
  'origin_country': 'us',
  'resource_key': 'german_credit_data_biased_training.csv',
  'rating': 0.0,
  'total_ratings': 0,
  'catalog_id': '32c875f2-f104-4801-9d5f-a0987098790c',
  'created': 1748009992882,
  'created_at': '2025-05-23T14:19:52Z',
  'owner_id': '1000331001',
  'size': 0,
  'version': 2.0,
  'asset_state': 'available',
  'asset_attributes': ['data_asset'],


In [10]:
from ibm_watsonx_ai.helpers import DataConnection, AssetLocation


credit_risk_conn = DataConnection(
    location=AssetLocation(asset_id=client.data_assets.get_id(asset_details))
)

training_data_reference = [credit_risk_conn]

### Optimizer configuration

Provide the input information for AutoAI optimizer:
- `name` - experiment name
- `prediction_type` - type of the problem
- `prediction_column` - target column name
- `scoring` - optimization metric
- `daub_include_only_estimators` - estimators which will be included during AutoAI training. More available estimators can be found in `experiment.ClassificationAlgorithms` enum

In [11]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, space_id=space_id)

pipeline_optimizer = experiment.optimizer(
    name="Credit Risk Bias detection in AutoAI",
    desc="Sample notebook",
    prediction_type=AutoAI.PredictionType.BINARY,
    prediction_column="Risk",
    scoring=AutoAI.Metrics.ROC_AUC_SCORE,
    include_only_estimators=[experiment.ClassificationAlgorithms.XGB],
)

<a id="run"></a>
## 3. Experiment run

Call the `fit()` method to trigger the AutoAI experiment. You can either use interactive mode (synchronous job) or background mode (asychronous job) by specifying `background_model=True`.

In [12]:
run_details = pipeline_optimizer.fit(
    training_data_reference=training_data_reference, background_mode=False
)

Training job 6331d3d1-a4e0-4a84-bbb0-2c7ab1a664ed completed: 100%|████████| [02:13<00:00,  1.34s/it]


In [13]:
pipeline_optimizer.get_run_status()

'completed'

You can list trained pipelines and evaluation metrics information in
the form of a Pandas DataFrame by calling the `summary()` method. 

In [14]:
summary = pipeline_optimizer.summary()
summary

,Enhancements,Estimator,training_roc_auc_(optimized),holdout_average_precision,holdout_log_loss,training_accuracy,holdout_roc_auc,training_balanced_accuracy,training_f1,holdout_precision,training_average_precision,training_log_loss,holdout_recall,training_precision,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1
Pipeline Name,,,,,,,,,,,,,,,,,,
Pipeline_2,HPO,XGBClassifier,0.857982,0.479641,0.422134,0.809573,0.815429,0.756301,0.865137,0.809249,0.920145,0.422384,0.936455,0.819500,0.810690,0.748227,0.916295,0.868217
Pipeline_3,"HPO, FE",XGBClassifier,0.860278,0.486580,0.445699,0.805854,0.813478,0.751840,0.862570,0.797143,0.921724,0.421039,0.933110,0.816658,0.797327,0.729889,0.914062,0.859784
Pipeline_4,"HPO, FE, HPO",XGBClassifier,0.861740,0.488660,0.454268,0.802384,0.812676,0.747935,0.860119,0.793003,0.922392,0.420921,0.909699,0.814289,0.781737,0.718183,0.911458,0.847352
Pipeline_5,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(XGBClassifier),0.861740,0.488660,0.454268,0.802384,0.812676,0.747935,0.860119,0.793003,0.922392,0.420921,0.909699,0.814289,0.781737,0.718183,0.911458,0.847352
Pipeline_1,,XGBClassifier,0.851228,0.466459,0.358376,0.798416,0.810301,0.748859,0.855829,0.839763,0.916677,0.435632,0.946488,0.817915,0.844098,0.793244,0.897693,0.889937


### Get selected pipeline model

Download pipeline model object from the AutoAI training job.

In [15]:
best_pipeline = pipeline_optimizer.get_pipeline()

<a id="bias"></a>
## 4. Bias detection and mitigation

The `fairness_info` dictionary contains some fairness-related metadata. The favorable and unfavorable label are values of the target class column that indicate whether the loan was granted or denied. A protected attribute is a feature that partitions the population into groups whose outcome should have parity. The credit-risk dataset has two protected attribute columns, sex and age. Each prottected attributes has privileged and unprivileged group.

Note that to use fairness metrics from lale with numpy arrays `protected_attributes.feature` need to be passed as index of the column in dataset, not as name.

In [16]:
fairness_info = {
    "favorable_labels": ["No Risk"],
    "protected_attributes": [
        {"feature": X.columns.get_loc("Sex"), "reference_group": ["male"]},
        {"feature": X.columns.get_loc("Age"), "reference_group": [[26, 40]]},
    ],
}
fairness_info

{'favorable_labels': ['No Risk'],
 'protected_attributes': [{'feature': 8, 'reference_group': ['male']},
  {'feature': 12, 'reference_group': [[26, 40]]}]}

### Calculate fairness metrics

We will calculate some model metrics. Accuracy describes how accurate is the model according to dataset. 
Disparate impact is defined by comparing outcomes between a privileged group and an unprivileged group, 
so it needs to check the protected attribute to determine group membership for the sample record at hand.
The third calculated metric takes the disparate impact into account along with accuracy. The best value of the score is 1.0.

In [17]:
import sklearn.metrics
from lale.lib.aif360 import disparate_impact, accuracy_and_disparate_impact

accuracy_scorer = sklearn.metrics.make_scorer(sklearn.metrics.accuracy_score)
print(f"accuracy {accuracy_scorer(best_pipeline, X_test.values, y_test.values):.1%}")
disparate_impact_scorer = disparate_impact(**fairness_info)
print(
    f"disparate impact {disparate_impact_scorer(best_pipeline, X_test.values, y_test.values):.2f}"
)
combined_scorer = accuracy_and_disparate_impact(**fairness_info)
print(
    f"accuracy and disparate impact metric {combined_scorer(best_pipeline, X_test.values, y_test.values):.2f}"
)

accuracy 79.2%
disparate impact 0.77
accuracy and disparate impact metric 0.78


### Mitigation

`Hyperopt` minimizes (`best_score` - `score_returned_by_the_scorer`), where `best_score` is an argument to `Hyperopt` and `score_returned_by_the_scorer` is the value returned by the scorer for each evaluation point. We will use the `Hyperopt` to tune hyperparametres of the AutoAI pipeline to get new and more fair model. 


In [18]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.tree import DecisionTreeClassifier as Tree
from sklearn.neighbors import KNeighborsClassifier as KNN
from lale.lib.lale import Hyperopt
from lale.lib.aif360 import FairStratifiedKFold
from lale import wrap_imported_operators

wrap_imported_operators()

In [19]:
prefix = best_pipeline.remove_last().freeze_trainable()
prefix.export_to_sklearn_pipeline()

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_140136462740672',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               12,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18,
                                                                                               19])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'float_int_num',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'c...
                                                                 ('numpyreplacemissingvalues',
                                                                  NumpyReplaceMissingValues(missing_values=[])),
                                                                 ('numimputer',
                                                                  NumImputer(missing_values=nan,
                                                                             strategy='median')),
                                                                 ('optstandardscaler',
                                                                  OptStandardScaler(use_scaler_flag=False)),
                                                                 ('float32_transform',
                                                                  float32_transform())]))])),
                ('numpypermutearray',
                 NumpyPermuteArray(axis=0,
                                   permutation_indices=[0, 1, 2, 3, 5, 6, 7, 8,
                                                        9, 10, 11,

In [20]:
fair_cv = FairStratifiedKFold(**fairness_info, n_splits=3)

new_pipeline = prefix >> (LR | Tree | KNN)
pipeline_fairer = new_pipeline.auto_configure(
    X_train.values,
    y_train.values,
    optimizer=Hyperopt,
    cv=fair_cv,
    max_evals=10,
    scoring=combined_scorer,
    best_score=1.0,
)

100%|██████████| 10/10 [00:13<00:00,  1.37s/trial, best loss: 0.15957762152193744]
1 out of 10 trials failed, call summary() for details.
Run with verbose=True to see per-trial exceptions.


As with any trained model, we can evaluate and visualize the result.

In [21]:
print(f"accuracy {accuracy_scorer(pipeline_fairer, X_test.values, y_test.values):.1%}")
print(
    f"disparate impact {disparate_impact_scorer(pipeline_fairer, X_test.values, y_test.values):.2f}"
)
print(
    f"accuracy and disparate impact metric {combined_scorer(pipeline_fairer, X_test.values, y_test.values):.2f}"
)

pipeline_fairer.export_to_sklearn_pipeline()

accuracy 73.0%
disparate impact 0.99
accuracy and disparate impact metric 0.86


Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_140136454738944',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               12,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18,
                                                                                               19])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'float_int_num',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'c...
                                                                  NumImputer(missing_values=nan,
                                                                             strategy='median')),
                                                                 ('optstandardscaler',
                                                                  OptStandardScaler(use_scaler_flag=False)),
                                                                 ('float32_transform',
                                                                  float32_transform())]))])),
                ('numpypermutearray',
                 NumpyPermuteArray(axis=0,
                                   permutation_indices=[0, 1, 2, 3, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19,
                                                        4])),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=86, weights='distance'))])

As the result demonstrates, the best model found by AI Automation
has lower accuracy and much better disparate impact as the one we saw
before. Also, it has tuned the repair level and
has picked and tuned a classifier. These results may vary by dataset and search space.

You can get source code of the created pipeline.

In [22]:
pipeline_fairer.pretty_print(ipython_display=True, show_imports=False)

```python
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[
        0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    ]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str", "float_int_num", "char_str", "char_str", "char_str",
        "char_str", "float_int_num", "char_str", "char_str", "float_int_num",
        "char_str", "float_int_num", "char_str", "char_str", "float_int_num",
        "char_str", "float_int_num", "char_str", "char_str",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), 100001, float("nan"), float("nan"), float("nan"),
        float("nan"), 100001, float("nan"), float("nan"), 100001,
        float("nan"), 100001, float("nan"), float("nan"), 100001,
        float("nan"), 100001, float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=float("nan"),
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
    encoding="ordinal",
    categories="auto",
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[4])
float_str2_float = FloatStr2Float(
    dtypes_list=["float_int_num"], missing_values_reference_list=[]
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    filling_values=float("nan"), missing_values=[]
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 4,
    ],
)
knn = KNN(n_neighbors=86, weights="distance")
pipeline = (
    (
        (
            numpy_column_selector_0
            >> compress_strings
            >> numpy_replace_missing_values_0
            >> numpy_replace_unknown_values
            >> boolean2float()
            >> cat_imputer
            >> cat_encoder
            >> float32_transform()
        )
        & (
            numpy_column_selector_1
            >> float_str2_float
            >> numpy_replace_missing_values_1
            >> num_imputer
            >> opt_standard_scaler
            >> float32_transform()
        )
    )
    >> ConcatFeatures()
    >> numpy_permute_array
    >> knn
)
```

<a id="scoring"></a>
## 5. Deploy and Score
In this section you will learn how to deploy and score Lale pipeline model using WML instance.

#### Custom software_specification

Created model is AutoAI model refined with Lale. We will create new software specification based on default Python 3.12 environment extended by  `autoai-libs` package.

In [23]:
base_sw_spec_id = client.software_specifications.get_id_by_name("runtime-25.1-py3.12")
print("Id of default Python 3.12 software specification is: ", base_sw_spec_id)

Id of default Python 3.12 software specification is:  f47ae1c3-198e-5718-b59d-2ea471561e9e


Create the file defining dependencies of package extension.

In [24]:
with open("requirements.txt", "w") as file:
    file.write("autoai-libs")

The `requirements.txt` file describes details of package extention. Now you need to store new package extention with `APIClient`.

In [25]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "Scikit with autoai-libs",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Package extension for autoai-libs",
    client.package_extensions.ConfigurationMetaNames.TYPE: "requirements_txt",
}

pkg_extn_details = client.package_extensions.store(
    meta_props=meta_prop_pkg_extn, file_path="requirements.txt"
)
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Creating package extensions
SUCCESS


Create new software specification and add created package extention to it. 

In [26]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "Mitigated AutoAI bases on scikit spec",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for scikt with autoai-libs",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        "guid": base_sw_spec_id
    },
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)


status = client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)

SUCCESS


You can get details of created software specification using `client.software_specifications.get_details(sw_spec_id)`

### Store the model

In [27]:
model_props = {
    client.repository.ModelMetaNames.NAME: "Fairer AutoAI model",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_1.6",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sw_spec_id,
}
feature_vector = list(X.columns)

In [28]:
published_model = client.repository.store_model(
    model=best_pipeline.export_to_sklearn_pipeline(),
    meta_props=model_props,
    training_data=X_train.values,
    training_target=y_train.values,
    feature_names=feature_vector,
    label_column_names=["Risk"],
)

In [29]:
published_model_id = client.repository.get_model_id(published_model)

### Deployment creation

In [30]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of fairer model",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '9bcbf6d8-c916-4124-a0c8-69d8861126b5' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='e39d4142-4ef7-4b9d-b76b-9767e5857afa'
-----------------------------------------------------------------------------------------------




In [31]:
deployment_id = client.deployments.get_id(created_deployment)

#### Deployment scoring 

You need to pass scoring values as input data if the deployed model. Use `client.deployments.score()` method to get predictions from deployed model. 

In [32]:
values = X_test.values

scoring_payload = {"input_data": [{"values": values[:5]}]}

In [33]:
predictions = client.deployments.score(deployment_id, scoring_payload)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['No Risk', [0.8500537872314453, 0.1499462127685547]],
    ['No Risk', [0.8045833110809326, 0.1954166740179062]],
    ['No Risk', [0.8392647504806519, 0.16073521971702576]],
    ['No Risk', [0.806576132774353, 0.19342388212680817]],
    ['No Risk', [0.8146058917045593, 0.18539410829544067]]]}]}

<a id="cleanup"></a>
## 6. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!

Check out used packeges domuntations:
- `ibm-watsonx-ai` [Online Documentation](https://ibm.github.io/watsonx-ai-python-sdk/index.html)
- `lale`: https://github.com/IBM/lale
- `aif360`: https://aif360.mybluemix.net/

### Authors 

**Dorota Lączak**, Software Engineer at watsonx.ai

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.